<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

1179. Reformat Department Table

https://leetcode.com/problems/reformat-department-table/description/

In [ ]:
import pandas as pd

def reformat_table(department: pd.DataFrame) -> pd.DataFrame:
    all_mths = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    pvt = department.pivot_table(index='id', columns='month', values='revenue').reindex(columns = all_mths)
    pvt.columns = [_+"_Revenue" for _ in pvt.columns]
    return pvt.reset_index()

175. Combine Two Tables

https://leetcode.com/problems/combine-two-tables/description/

In [3]:
data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})
data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

```sql
SELECT
  p.firstName,
  p.lastName,
  a.city,
  a.state
FROM person p
LEFT JOIN Address a
ON p.personId = a.personId

```

In [6]:
person.merge(address, how='left', on='personId')[['firstName', 'lastName', 'city','state' ]]

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


181. Employees Earning More Than Their Managers

https://leetcode.com/problems/employees-earning-more-than-their-managers/description/

In [25]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

```sql
SELECT e2.name as  Employee FROM Employee e1
INNER JOIN Employee e2 on e1.id = e2.managerId
WHERE e1.salary < e2.salary
```


In [26]:
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(employee, how='inner', left_on='id', right_on='managerId')
    merged = merged[merged.salary_x < merged.salary_y][['name_y']].rename(columns={'name_y':'Employee'})
    return merged

182. Duplicate Emails

https://leetcode.com/problems/duplicate-emails/description/

```sql
SELECT email as "Email" FROM Person
GROUP BY email
HAVING count(email) > 1
```

In [28]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})

In [51]:
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    return person.groupby('email', as_index=False).count().query("id > 1", engine="python")[['email']]

183. Customers Who Never Order

https://leetcode.com/problems/customers-who-never-order/description/

```sql
SELECT name as "Customers" FROM Customers c WHERE c.id NOT IN (SELECT customerId FROM Orders )
```

In [52]:
data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=['id', 'customerId']).astype({'id':'Int64', 'customerId':'Int64'})

In [64]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return customers[~customers['id']\
                     .isin(orders.customerId)][['name']]\
                     .rename({'name': 'Customers'}, axis=1)

196. Delete Duplicate Emails

https://leetcode.com/problems/delete-duplicate-emails/description/

```sql
DELETE FROM Person WHERE id in (
SELECT p1.id FROM Person p1
JOIN Person p2 ON p1.email = p2.email and p1.id > p2.id)
```


In [67]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})

In [81]:
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values(by='id',ascending=True,inplace=True)
    person.drop_duplicates(subset='email', keep='first', inplace=True)

197. Rising Temperature

https://leetcode.com/problems/rising-temperature/description/

```sql

SELECT id as Id FROM (
SELECT *,
lag(temperature, 1) OVER (ORDER BY recordDate) as tlag,
lag(recordDate , 1) OVER (ORDER BY recordDate) as dlag
FROM Weather) t
WHERE temperature - tlag > 0 and recordDate - dlag = 1
```

In [94]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [117]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather = weather.sort_values(by='recordDate')
    df = weather[(weather['temperature'].diff()>0) & (weather['recordDate'].diff().dt.days == 1)]
    return df[['id']]

,id
1,2
3,4
